<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/dhn/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install all the dependencies. Place any new installables into requirements.txt file.
pip -r requirements.txt

In [ ]:
import torch

In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Select an audio file and read it:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = ds[0]["audio"]

# Load the Whisper model with SDPA attention
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en", attn_implementation="sdpa")

# Enable static cache and compile the forward pass
model.generation_config.cache_implementation = "static"
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

# Use the model and processor to transcribe the audio:
input_features = processor(
    audio_sample["array"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt"
).input_features

# Compile the forward pass
for _ in range(2):
    model.generate(input_features)

# Generate token ids using compiled graph (fast!)
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

transcription[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The 'batch_size' attribute of StaticCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.'

In [ ]:
from transformers import AutoProcessor, Wav2Vec2BertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")
model = Wav2Vec2BertForCTC.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

librispeech_asr_demo.py:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

17.04

In [ ]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

transcription = whisper("/content/drive/MyDrive/Colab Notebooks/EarningsCall.wav", return_timestamps=True)

print(transcription["text"])

Device set to use cuda:0


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/EarningsCall.wav'

In [5]:
import torch
from transformers import pipeline
import gradio as gr

def Speech2TextWhisperLarge(audioFile) :
    whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")
    transcription = whisper("/content/drive/MyDrive/Colab Notebooks/EarningsCall.wav", return_timestamps=True)
    #print(transcription["text"])
    return transcription



demo = gr.Interface(
        fn=Speech2TextWhisperLarge,
        title="Speech to Text",
        inputs=["file",
              #  gr.Radio(choices=["Yes", "No"], label="Task", value="Generate"),
                ],
       outputs=["text"])

demo.launch(debug=True, show_error=True)

# demo = gr.Interface(
#         fn=regression_transformer,
#         title="Regression Transformer",
#         inputs=[
#             gr.Dropdown(rt_names, label="Algorithm version", value="Molecules: Qed"),
#             gr.Radio(choices=["Predict", "Generate"], label="Task", value="Generate"),
#             gr.Textbox(
#                 label="Input", placeholder="CC(C#C)N(C)C(=O)NC1=CC=C(Cl)C=C1", lines=1
#             ),
#             gr.Slider(
#                 minimum=1, maximum=50, value=10, label="Number of samples", step=1
#             ),
#             gr.Radio(choices=["Sample", "Greedy"], label="Search", value="Sample"),
#             gr.Slider(minimum=0.5, maximum=2, value=1, label="Decoding temperature"),
#             gr.Slider(minimum=5, maximum=100, value=30, label="Tolerance", step=1),
#             gr.Radio(choices=[True, False], label="Sampling Wrapper", value=True),
#             gr.Slider(minimum=0, maximum=1, value=0.5, label="Fraction to mask"),
#             gr.Textbox(label="Property goal", placeholder="<qed>:0.75", lines=1),
#             gr.Textbox(label="Tokens to mask", placeholder="N, C", lines=1),
#             gr.Textbox(
#                 label="Substructures to mask", placeholder="C(=O), C#C", lines=1
#             ),
#             gr.Textbox(
#                 label="Substructures to keep", placeholder="C1=CC=C(Cl)C=C1", lines=1
#             ),
#         ],
#         outputs=gr.HTML(label="Output"),
#         article=article,
#         description=description,
#         examples=examples.values.tolist(),
#     )
#     demo.launch(debug=True, show_error=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dd5b76e1970f76b9dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dd5b76e1970f76b9dd.gradio.live


In [ ]:
print(transcription["text"])

 Good afternoon, everyone, and welcome to FinTech Plus Sync's second quarter 2023 earnings call. I'm John Doe, CEO of FinTech Plus. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from our scalable business model. Our EBITDA has surged to $37.5 million, translating to a remarkable 30% EBITDA margin. Our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in Q2 2022. Our total addressable market has grown substantially, thanks to the expansion of our high-yield savings product line and the new RoboAdvisor platform. We've been diversifying our asset-backed securities portfolio, investing heavily in collateralized debt obligations and residential mortgage-backed securities. We've also invested $25 million in AAA-rated corporate bonds, enhancing our risk-adjusted returns. As for our balance sheet, total assets reached $1.5 

In [2]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
from pydub import AudioSegment
import torch
from transformers import pipeline

# Load Whisper model
whisper = pipeline(
    "automatic-speech-recognition",
    "openai/whisper-large-v3",
    torch_dtype=torch.float16,
    device="cuda:0"
)

# Load the audio file
audio = AudioSegment.from_wav("/content/drive/MyDrive/Colab Notebooks/EarningsCall.wav")

# Define chunk length (e.g., 25 seconds)
chunk_length_ms = 25000  # 25 seconds
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

# Save & Transcribe each chunk
transcriptions = []
for idx, chunk in enumerate(chunks):
    chunk.export(f"chunk_{idx}.wav", format="wav")
    result = whisper(f"chunk_{idx}.wav")  # Transcribe
    transcriptions.append(result["text"])

# Combine the transcriptions
full_transcription = " ".join(transcriptions)
print(full_transcription)


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: T

 Good afternoon, everyone, and welcome to FinTech Plus Sync's second quarter 2023 earnings call. I'm John Doe, CEO of FinTech Plus. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from  scalable business model. Our EBITDA has surged to $37.5 million, translating to a remarkable 30% EBITDA margin. Our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in Q2 2022. Our total addressable market has grown substantially thanks to the expansion of our high-yield savings  product line and the new RoboAdvisor platform. We've been diversifying our asset-backed securities portfolio, investing heavily in collateralized debt obligations and residential mortgage-backed securities. We've also invested $25 million in AAA-rated corporate bonds, enhancing our risk-adjusted returns. As for our balance sheet,  Total assets reached $1.5 bi

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

print(tokenizer.backend_tokenizer.normalizer.normalize_str(full_transcription))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

 good afternoon, everyone, and welcome to fintech plus sync's second quarter 2023 earnings call. i'm john doe, ceo of fintech plus. we've had a stellar q2 with a revenue of $125 million, a 25% increase year over year. our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from  scalable business model. our ebitda has surged to $37.5 million, translating to a remarkable 30% ebitda margin. our net income for the quarter rose to $16 million, which is a noteworthy increase from $10 million in q2 2022. our total addressable market has grown substantially thanks to the expansion of our high-yield savings  product line and the new roboadvisor platform. we've been diversifying our asset-backed securities portfolio, investing heavily in collateralized debt obligations and residential mortgage-backed securities. we've also invested $25 million in aaa-rated corporate bonds, enhancing our risk-adjusted returns. as for our balance sheet,  total assets reached $1.5 bi

In [ ]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(full_transcription)

[('Good', (1, 5)),
 ('afternoon', (6, 15)),
 (',', (15, 16)),
 ('everyone', (17, 25)),
 (',', (25, 26)),
 ('and', (27, 30)),
 ('welcome', (31, 38)),
 ('to', (39, 41)),
 ('FinTech', (42, 49)),
 ('Plus', (50, 54)),
 ('Sync', (55, 59)),
 ("'", (59, 60)),
 ('s', (60, 61)),
 ('second', (62, 68)),
 ('quarter', (69, 76)),
 ('2023', (77, 81)),
 ('earnings', (82, 90)),
 ('call', (91, 95)),
 ('.', (95, 96)),
 ('I', (97, 98)),
 ("'", (98, 99)),
 ('m', (99, 100)),
 ('John', (101, 105)),
 ('Doe', (106, 109)),
 (',', (109, 110)),
 ('CEO', (111, 114)),
 ('of', (115, 117)),
 ('FinTech', (118, 125)),
 ('Plus', (126, 130)),
 ('.', (130, 131)),
 ('We', (132, 134)),
 ("'", (134, 135)),
 ('ve', (135, 137)),
 ('had', (138, 141)),
 ('a', (142, 143)),
 ('stellar', (144, 151)),
 ('Q2', (152, 154)),
 ('with', (155, 159)),
 ('a', (160, 161)),
 ('revenue', (162, 169)),
 ('of', (170, 172)),
 ('$', (173, 174)),
 ('125', (174, 177)),
 ('million', (178, 185)),
 (',', (185, 186)),
 ('a', (187, 188)),
 ('25', (189, 191

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(full_transcription)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[('ĠGood', (0, 5)),
 ('Ġafternoon', (5, 15)),
 (',', (15, 16)),
 ('Ġeveryone', (16, 25)),
 (',', (25, 26)),
 ('Ġand', (26, 30)),
 ('Ġwelcome', (30, 38)),
 ('Ġto', (38, 41)),
 ('ĠFinTech', (41, 49)),
 ('ĠPlus', (49, 54)),
 ('ĠSync', (54, 59)),
 ("'s", (59, 61)),
 ('Ġsecond', (61, 68)),
 ('Ġquarter', (68, 76)),
 ('Ġ2023', (76, 81)),
 ('Ġearnings', (81, 90)),
 ('Ġcall', (90, 95)),
 ('.', (95, 96)),
 ('ĠI', (96, 98)),
 ("'m", (98, 100)),
 ('ĠJohn', (100, 105)),
 ('ĠDoe', (105, 109)),
 (',', (109, 110)),
 ('ĠCEO', (110, 114)),
 ('Ġof', (114, 117)),
 ('ĠFinTech', (117, 125)),
 ('ĠPlus', (125, 130)),
 ('.', (130, 131)),
 ('ĠWe', (131, 134)),
 ("'ve", (134, 137)),
 ('Ġhad', (137, 141)),
 ('Ġa', (141, 143)),
 ('Ġstellar', (143, 151)),
 ('ĠQ', (151, 153)),
 ('2', (153, 154)),
 ('Ġwith', (154, 159)),
 ('Ġa', (159, 161)),
 ('Ġrevenue', (161, 169)),
 ('Ġof', (169, 172)),
 ('Ġ$', (172, 174)),
 ('125', (174, 177)),
 ('Ġmillion', (177, 185)),
 (',', (185, 186)),
 ('Ġa', (186, 188)),
 ('Ġ25', (188, 191

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

print(summarizer(full_transcription, max_length=130, min_length=30, do_sample=False))

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


[{'summary_text': "John Doe, CEO of FinTech Plus. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from  scalable business model. Our forecast for the coming quarter is positive. We expect revenue to be around 135 million and 8% quarter over quarter growth."}]


In [ ]:
summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)
print(summary)

[{'summary_text': "John Doe, CEO of FinTech Plus. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year over year. Our gross profit margin stands at a solid 58%, due in part to cost efficiencies gained from  scalable business model. Our forecast for the coming quarter is positive. We expect revenue to be around 135 million and 8% quarter over quarter growth."}]


In [ ]:

# # Function to extract action items
# import re

# def extract_action_items(full_transcription):
#     action_keywords = ["must", "should", "need to", "required", "ensure", "action item", "task", "responsible for", "deadline"]
#     sentences = re.split(r'(?<=[.!?])\s+', full_transcription)
#     action_items = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in action_keywords)]
#     return action_items

# # Extract action items from the transcribed text
# action_items = extract_action_items(transcription[0])

# # Display extracted action items
# print("Extracted Action Items:")
# for idx, item in enumerate(action_items, start=1):
#     print(f"{idx}. {item}")


In [ ]:
# # Function to extract action items
# import re
# print(full_transcription)
# def extract_action_items(full_transcription):
#     action_keywords = ["must", "should", "need to", "Investing", "Expanding", "Diversifying", "Enhancing"]
#     sentences = re.split(r'(?<=[.!?])\s+', full_transcription)
#     action_items = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in action_keywords)]
#     return action_items

# # Extract action items from the transcribed text
# # Access the 'text' key of the transcription dictionary
# action_items = extract_action_items(full_transcription)

# # Display extracted action items
# print("Extracted Action Items:")
# for idx, item in enumerate(action_items, start=1):
#     print(f"{idx}. {item}")

In [ ]:
import re

def extract_action_items(full_transcription):
    """
    Extracts action items from the given transcribed text by identifying sentences with action-oriented keywords.
    """
    # Keywords that indicate action items
    action_keywords = ["must", "should", "need to", "Investing", "Expanding", "Diversifying", "Enhancing", "revenue", "risk", "loss"]

    # Split text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', full_transcription)

    # Extract sentences that contain action-related words
    action_items = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in action_keywords)]

    return action_items

# Example transcribed text (Replace this with your actual transcription)
# full_transcription1 = """
# The team needs to complete the project report by next Monday.
# John is responsible for finalizing the budget.
# Ensure that all client feedback is incorporated before submission.
# We had a general discussion about the upcoming product launch.
# Sarah should follow up with the vendor for the new software license.
# """

# Extract action items
action_items = extract_action_items(full_transcription)

# Print extracted action items
print("Extracted Action Items:")
for idx, item in enumerate(action_items, start=1):
    print(f"{idx}. {item}")


Extracted Action Items:
1. We've had a stellar Q2 with a revenue of $125 million, a 25% increase year over year.
2. We've also invested $25 million in AAA-rated corporate bonds, enhancing our risk-adjusted returns.
3. In terms of risk management, we have a value at risk model in place with a 99% confidence level indicating that our maximum loss will not exceed  5 million in the next trading day.
4. We expect revenue to be around 135 million and 8% quarter over quarter growth, driven primarily by our cutting edge blockchain solutions and AI technologies.


In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
load_recall = evaluate.load("recall")
load_f1 = evaluate.load("f1")

In [ ]:
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Download the 'punkt' and 'punkt_tab' resources
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#Evaluating metrics with full_transcription

import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

nltk.download('punkt')

# Sample reference summary (manually written or an official meeting summary)
#reference_summary_sample = ["The team must complete the project by next week. The budget needs approval."]

# Transcribed text (output from Whisper model)
#full_transcription_sample = "The project completion deadline is next week. The team is responsible. Budget needs to be checked."

# Tokenize reference and generated text
reference_tokens = [nltk.word_tokenize(summary[0]['summary_text'])]
transcribed_tokens = nltk.word_tokenize(full_transcription)

# Compute BLEU Score
bleu_score = sentence_bleu(reference_tokens, transcribed_tokens, smoothing_function=SmoothingFunction().method1)

# Compute ROUGE Score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(summary[0]['summary_text'], full_transcription)

# Convert text to binary vectors for metric calculations
ref_words = set(reference_tokens[0])
trans_words = set(transcribed_tokens)
all_words = list(ref_words.union(trans_words))

y_true = [1 if word in ref_words else 0 for word in all_words]
y_pred = [1 if word in trans_words else 0 for word in all_words]

# Compute F1 Score
f1 = f1_score(y_true, y_pred)

# Compute Recall
recall = recall_score(y_true, y_pred)

# Compute Precision
precision = precision_score(y_true, y_pred)

# Compute Accuracy
accuracy = accuracy_score(y_true, y_pred)

print("\n\n")
# Print Evaluation Metrics
print(f"\nBLEU Score: {bleu_score:.4f}")
print(f"\nPrecision Score: {precision:.4f}")
print(f"\nAccuracy Score: {accuracy:.4f}")
print(f"\nF1 Score: {f1:.4f}")
print(f"\nROUGE Scores: {rouge_scores}")
print(f"\nRecall Score: {recall:.4f}")






BLEU Score: 0.1791

Precision Score: 0.2468

Accuracy Score: 0.2468

F1 Score: 0.3958

ROUGE Scores: {'rouge1': Score(precision=0.1741424802110818, recall=1.0, fmeasure=0.2966292134831461), 'rouge2': Score(precision=0.17195767195767195, recall=1.0, fmeasure=0.29345372460496616), 'rougeL': Score(precision=0.1741424802110818, recall=1.0, fmeasure=0.2966292134831461)}

Recall Score: 1.0000


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
